In [18]:
using GLMakie, LinearAlgebra

# TODO: Make border limits for boids
# TODO: Make boids not collided
# TODO: Maybeeeeee try make .exe 

# Include file with movement algorithm
include("boids.jl")

# Make boids
width = height = 200;
numBoids = 400
flock = [Boid(i, rand(-width:width, 3), randn(3).*3, randn(3).*2) for i=1:numBoids]
points = Observable([Point3f(boid.position) for boid in flock])

# Create figure
fig = Figure()

# Create 3d axis
ax = Axis3(fig[1, 1], aspect = (1, 1, 1),
        title = "Flocking simulation")
        ax.perspectiveness = 0

# Set axis limits
limits!(ax, -2*width, 2*width, -2*width, 2*width, -2*width, 2*width)

# Create sliders layout
lsgrid = labelslidergrid!(fig,
    ["align", "separation", "cohesion"],
    Ref(LinRange(0.01:0.01:3));
    formats = [x -> "$(round(x, digits = 2))"],
    labelkw = Dict([(:textsize, 20)]),
    sliderkw = Dict([(:linewidth, 10)]),
    valuekw = Dict([(:textsize, 20)])
)

# Create sliders grid
sl_sublayout = GridLayout(height=50)
fig[2, 1] = sl_sublayout
fig[2, 1] = lsgrid.layout

# Set initial parameters
set_close_to!(lsgrid.sliders[1], 0.5)
set_close_to!(lsgrid.sliders[2], 1.0)
set_close_to!(lsgrid.sliders[3], 0.6)

# Update parameters values
alignValue = lsgrid.sliders[1].value
separationValue = lsgrid.sliders[2].value
cohesionValue = lsgrid.sliders[3].value

# Create buttons grid
fig[3, 1] = buttongrid = GridLayout(height=100)

labels = ["Run / Pause", "Step", "Restart"]
button_count = 3

# Create buttons
buttons = buttongrid[3, 1:button_count] = [
    Button(fig, label = l, height = 35, width = 150, textsize = 20)
    for l in labels
]

# Run button
isrunning = Observable(false)
on(buttons[1].clicks) do clicks; isrunning[] = !isrunning[]; end

on(buttons[1].clicks) do clicks
    @async while isrunning[]
        for boid in flock
            fly!(boid, flock, alignValue[], separationValue[], cohesionValue[])
        end
        
        points[] = [Point3f(boid.position) for boid in flock]
        sleep(0.00001)
    end
end

# Step button
on(buttons[2].clicks) do click
    for boid in flock
        fly!(boid, flock, alignValue[], separationValue[], cohesionValue[])
    end
    
    points[] = [Point3f(boid.position) for boid in flock]
end

# Reset button
on(buttons[3].clicks) do click
    for boid in flock
        boid.position = rand(-width:width, 3)
    end
    points[] = [Point3f(boid.position) for boid in flock]
end

# scatter!(ax, points, strokewidth=1, color = :black)
scatter!(ax, points, markersize=1000, color = :black)
display(fig)

GLMakie.Screen(...)